Это ноутбук для подготовки данных и автоматического написания скриптов для запуска инструментов-предикторов

In [222]:
from typing import List, Dict, Optional, Literal, Any
from dataclasses import dataclass

import numpy as np
import pandas as pd

import os
import requests
import tqdm

import uuid
import json

import prody

In [248]:
@dataclass
class Chain:
    sequence: str
    molecule_type: Literal["protein", "dna", "rna"]
    pdb_id: Optional[str]

    def __post_init__(self):
        self.pdb_id = self.pdb_id or "DEFAULT_PDB_ID"
        self.pdb_id = self.pdb_id.upper()

    # Функция соавторства GigaChat
    @classmethod
    def from_prody_chain(cls, chain: prody.Chain, pdb_id: Optional[str] = None):
        """
        Пока что без поддержки модифицированных остатков/нуклеотидов
        """

        protein_residues = {'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D', 'CYS': 'C', 'GLN': 'Q', 'GLU': 'E',
                            'GLY': 'E', 'HIS': 'H', 'ILE': 'I', 'LEU': 'L', 'LYS': 'K', 'MET': 'M', 'PHE': 'F',
                            'PRO': 'P', 'SER': 'S', 'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V',
                            # Нетрадиционные ребята, но их просто заменяем
                            'MSE': 'M', 'PTR': 'Y', 'SEP': 'S', 'CME': 'C', 'TPO': 'T', 'CSO': 'C'}
        dna_residues = {'DA': 'A', 'DC': 'C', 'DG': 'G', 'DT': 'T', 'DU': 'U', 'GTP': 'G', 'GDP': 'G'}
        rna_residues = {'A': 'A', 'C': 'C', 'G': 'G', 'U': 'U', 'T': 'T', 'GTP': 'G', 'GDP': 'G'}

        residues = (
            chain.select("name CA")  # protein
            or chain.select("name N1")  # NA
        )

        if not residues:
            return None

        residues = residues.select("protein or nucleic")

        if not residues:
            return None

        resnames = residues.getResnames()
        unique_resnames = set(resnames)

        if unique_resnames.issubset(set(protein_residues.keys())):
            return cls(
                sequence="".join(protein_residues[resname] for resname in resnames),
                molecule_type="protein",
                pdb_id=pdb_id,
            )

        if unique_resnames.issubset(set(dna_residues.keys())):
            return cls(
                sequence="".join(dna_residues[resname] for resname in resnames),
                molecule_type="dna",
                pdb_id=pdb_id,
            )

        if unique_resnames.issubset(set(rna_residues.keys())):
            return cls(
                sequence="".join(rna_residues[resname] for resname in resnames),
                molecule_type="rna",
                pdb_id=pdb_id,
            )

        # print(pdb_id, unique_resnames - set(protein_residues.keys()) - set(dna_residues.keys()) - set(rna_residues.keys()))
        return None

    def to_fasta_string_chai(self) -> str:
        """
        >rna|4N2Q_rna
        """
        # Chai ругается, если имена цепей не уникальные
        header = ">" + self.molecule_type + "|" + (self.pdb_id or "DEFAULT_PDB_ID") + "_" + self.molecule_type + "_" + str(uuid.uuid4())
        return header + "\n" + self.sequence + "\n"

    def to_fasta_string_boltz(self, index=0) -> str:
        """
        >r|rna
        """
        # У Больца странный баг, что не однобуквенное название цепи вызывает KeyError
        # Чтобы различать цепи, будем подавать на вход номер этой цепи в структуре (среди этого типа молекул),
        # и по нему брать букву - заглавную для белка и малую для НК (жесть костыль, но а как)
        alpha = ["a", "b", "c", "d", "e", "f", "g", "h", "j", "k", "l", "m", "n", "o", 
                 "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
        chain_id = alpha[index].lower() if self.molecule_type in {"rna", "dna"} else alpha[index].upper()
        header = ">" + chain_id + "|" + self.molecule_type
        # Указываем, что нет выравнивания, если белок
        if self.molecule_type == "protein":
            header += "|empty"
        return header + "\n" + self.sequence + "\n"

    def to_dict_protenix(self, msa_path=None) -> Dict[str, Any]:
        outter_key_dict = {
            "protein": "proteinChain",
            "rna": "rnaSequence",
            "dna": "dnaSequence"
        }
        result = {
            outter_key_dict[self.molecule_type]: {
                "sequence": self.sequence,
                "count": 1
            }
        }
        if self.molecule_type == "protein":
            uid = str(uuid.uuid4())
            a3m_string = ">query\n" + self.sequence
            os.makedirs(f"{msa_path}/{uid}", exist_ok=True)
            with open(f"{msa_path}/{uid}/pairing.a3m", "w") as f:
                f.write(a3m_string)
            with open(f"{msa_path}/{uid}/non_pairing.a3m", "w") as f:
                f.write(a3m_string)
            result[outter_key_dict[self.molecule_type]]["msa"] = {
                "precomputed_msa_dir": f"{msa_path}/{uid}",
                "pairing_db": "uniref100"
            }
        return result


In [249]:
fasta_strings_chai = {}
fasta_strings_boltz = {}
unluck_cnt = 0

pdb_id_to_na_type = {}

os.makedirs(f"/mnt/storage/vladislove2020/NP_benchmark/protenix/bench_3/workdir", exist_ok=True)

for filename in tqdm.tqdm(os.listdir("/mnt/storage/vladislove2020/masif_na/PN")):
    try:
        struct = prody.parsePDB(f"/mnt/storage/vladislove2020/masif_na/PN/{filename}")
    except:
        continue
    pdb_id = filename.split(".")[0]
    chains = []
    for chain in struct.iterChains():
        chain_datacls = Chain.from_prody_chain(chain=chain, pdb_id=pdb_id)
        if not chain_datacls:
            unluck_cnt += 1
            continue
        chains.append(
            chain_datacls
        )
        
    # DNA-RNA дуплексы не учитываем при разбивке по типу NA
    if {chain.molecule_type for chain in chains} == {"protein", "rna"}:
        pdb_id_to_na_type[pdb_id] = "rna"
    elif {chain.molecule_type for chain in chains} == {"protein", "dna"}:
        pdb_id_to_na_type[pdb_id] = "dna"
    elif {chain.molecule_type for chain in chains} == {"protein", "dna", "rna"}:
        pdb_id_to_na_type[pdb_id] = "rna_dna"
    # else:
    #     print(pdb_id, {chain.molecule_type for chain in chains})

    if len(chains) > 24:
        unluck_cnt += 1
        # print(pdb_id)
        continue
    if set([chain.molecule_type for chain in chains]) not in [{"dna", "protein"}, {"rna", "protein"}]:
        unluck_cnt += 1
        continue
    fasta_strings_chai[pdb_id] = "".join([chain.to_fasta_string_chai() for chain in chains])
    fasta_strings_boltz[pdb_id] = "".join([chain.to_fasta_string_boltz(index=i) for i, chain in enumerate(chains)])

    os.makedirs(f"/mnt/storage/vladislove2020/NP_benchmark/protenix/bench_3/workdir/{pdb_id}", exist_ok=True)
    os.makedirs(f"/mnt/storage/vladislove2020/NP_benchmark/protenix/bench_3/workdir/{pdb_id}/msa", exist_ok=True)
    with open(f"/mnt/storage/vladislove2020/NP_benchmark/protenix/bench_3/workdir/{pdb_id}/input.json", "w") as js:
        json.dump(
            obj=[
                {
                    "sequences": [
                        chain.to_dict_protenix(
                            msa_path=f"/mnt/storage/vladislove2020/NP_benchmark/protenix/bench_3/workdir/{pdb_id}/msa"
                        ) for chain in chains
                    ],
                    "name": pdb_id
                }
            ],
            fp=js,
            ensure_ascii=False,
            indent=4
        )

with open("/mnt/storage/vladislove2020/NP_benchmark/pdb_id_to_na_type.json", "w") as js:
    json.dump(
        fp=js,
        obj=pdb_id_to_na_type,
        indent=4,
        ensure_ascii=False
    )

100%|██████████| 1054/1054 [01:37<00:00, 10.79it/s]


In [205]:
unluck_cnt

114

In [206]:
len(fasta_strings_boltz)

1020

In [207]:
fasta_dir_path_chai = "/mnt/storage/vladislove2020/NP_benchmark/chai-1/fasta_inputs_unmodified_only"
os.makedirs(fasta_dir_path_chai, exist_ok=True)

for pdb_id, fasta_string in fasta_strings_chai.items():
    with open(f"{fasta_dir_path_chai}/{pdb_id}.fasta", "w") as file:
        file.write(fasta_string)

        
fasta_dir_path_boltz = "/mnt/storage/vladislove2020/NP_benchmark/boltz-1/fasta_inputs_unmodified_only"
os.makedirs(fasta_dir_path_boltz, exist_ok=True)

for pdb_id, fasta_string in fasta_strings_boltz.items():
    with open(f"{fasta_dir_path_boltz}/{pdb_id}.fasta", "w") as file:
        file.write(fasta_string)


# fasta_dir_path_rosettafold2na = "/mnt/storage/vladislove2020/NP_benchmark/RosettaFold2NA/fasta_inputs_unmodified_only"
# os.makedirs(fasta_dir_path_rosettafold2na, exist_ok=True)

# for pdb_id, fasta_strings in fasta_strings_rosettafold2na.items():
#     for 
#     with open(f"{fasta_dir_path_rosettafold2na}/{pdb_id}.fasta", "w") as file:
#         file.write(fasta_string)

In [208]:
### Chai-1 script

bench_path = "/mnt/storage/vladislove2020/NP_benchmark/chai-1/bench_3"
os.makedirs(bench_path, exist_ok=True)
os.makedirs(f"{bench_path}/results", exist_ok=True)

script = "#!/usr/bin/bash\n"

for filename in os.listdir(fasta_dir_path_chai):
    pdb_id = filename.split(".")[0]
    os.makedirs(f"{bench_path}/results/{pdb_id}", exist_ok=True)
    script += f"chai-lab fold {fasta_dir_path_chai}/{filename} results/{pdb_id}\n"

with open(f"{bench_path}/chai.sh", "w") as file:
    file.write(script)

In [209]:
!head -8 /mnt/storage/vladislove2020/NP_benchmark/chai-1/bench_3/chai.sh

#!/usr/bin/bash
chai-lab fold /mnt/storage/vladislove2020/NP_benchmark/chai-1/fasta_inputs_unmodified_only/1m8x.fasta results/1m8x
chai-lab fold /mnt/storage/vladislove2020/NP_benchmark/chai-1/fasta_inputs_unmodified_only/6u7t.fasta results/6u7t
chai-lab fold /mnt/storage/vladislove2020/NP_benchmark/chai-1/fasta_inputs_unmodified_only/5h1k.fasta results/5h1k
chai-lab fold /mnt/storage/vladislove2020/NP_benchmark/chai-1/fasta_inputs_unmodified_only/5gnj.fasta results/5gnj
chai-lab fold /mnt/storage/vladislove2020/NP_benchmark/chai-1/fasta_inputs_unmodified_only/6blw.fasta results/6blw
chai-lab fold /mnt/storage/vladislove2020/NP_benchmark/chai-1/fasta_inputs_unmodified_only/4iqr.fasta results/4iqr
chai-lab fold /mnt/storage/vladislove2020/NP_benchmark/chai-1/fasta_inputs_unmodified_only/3l2c.fasta results/3l2c


In [210]:
!chmod +x /mnt/storage/vladislove2020/NP_benchmark/chai-1/bench_3/chai.sh

Launch from tmux with conda from /home/domain/rubeckaya/miniconda3/envs/chai

In [211]:
### Boltz-1 script

bench_path = "/mnt/storage/vladislove2020/NP_benchmark/boltz-1/bench_3"
os.makedirs(bench_path, exist_ok=True)
os.makedirs(f"{bench_path}/results", exist_ok=True)

script = "#!/usr/bin/bash\n"

for filename in os.listdir(fasta_dir_path_boltz):
    pdb_id = filename.split(".")[0]
    os.makedirs(f"{bench_path}/results/{pdb_id}", exist_ok=True)
    script += f"boltz predict {fasta_dir_path_boltz}/{filename} --out_dir results/{pdb_id} --output_format pdb\n"

with open(f"{bench_path}/boltz.sh", "w") as file:
    file.write(script)

In [212]:
!head -8 /mnt/storage/vladislove2020/NP_benchmark/boltz-1/bench_3/boltz.sh

#!/usr/bin/bash
boltz predict /mnt/storage/vladislove2020/NP_benchmark/boltz-1/fasta_inputs_unmodified_only/3uzt.fasta --out_dir results/3uzt --output_format pdb
boltz predict /mnt/storage/vladislove2020/NP_benchmark/boltz-1/fasta_inputs_unmodified_only/1vfc.fasta --out_dir results/1vfc --output_format pdb
boltz predict /mnt/storage/vladislove2020/NP_benchmark/boltz-1/fasta_inputs_unmodified_only/3adb.fasta --out_dir results/3adb --output_format pdb
boltz predict /mnt/storage/vladislove2020/NP_benchmark/boltz-1/fasta_inputs_unmodified_only/5d4s.fasta --out_dir results/5d4s --output_format pdb
boltz predict /mnt/storage/vladislove2020/NP_benchmark/boltz-1/fasta_inputs_unmodified_only/6cnq.fasta --out_dir results/6cnq --output_format pdb
boltz predict /mnt/storage/vladislove2020/NP_benchmark/boltz-1/fasta_inputs_unmodified_only/5wwe.fasta --out_dir results/5wwe --output_format pdb
boltz predict /mnt/storage/vladislove2020/NP_benchmark/boltz-1/fasta_inputs_unmodified_only/1u1r.fasta --out

In [213]:
!chmod +x /mnt/storage/vladislove2020/NP_benchmark/boltz-1/bench_3/boltz.sh

Launch from tmux with conda from /home/domain/vladislove2020/miniconda3/envs/boltz

In [250]:
# Protenix script
bench_path = "/mnt/storage/vladislove2020/NP_benchmark/protenix/bench_3"
os.makedirs(f"{bench_path}/results", exist_ok=True)

script = "#!/usr/bin/bash\n"

for filename in os.listdir(fasta_dir_path_boltz):
    pdb_id = filename.split(".")[0]
    os.makedirs(f"{bench_path}/results/{pdb_id}", exist_ok=True)
    script += f"protenix predict --input {bench_path}/workdir/{pdb_id}/input.json --out_dir results/{pdb_id}\n"

with open(f"{bench_path}/protenix.sh", "w") as file:
    file.write(script)

In [251]:
!chmod +x /mnt/storage/vladislove2020/NP_benchmark/protenix/bench_3/protenix.sh

In [252]:
!head -8 /mnt/storage/vladislove2020/NP_benchmark/protenix/bench_3/protenix.sh

#!/usr/bin/bash
protenix predict --input /mnt/storage/vladislove2020/NP_benchmark/protenix/bench_3/workdir/3uzt/input.json --out_dir results/3uzt
protenix predict --input /mnt/storage/vladislove2020/NP_benchmark/protenix/bench_3/workdir/1vfc/input.json --out_dir results/1vfc
protenix predict --input /mnt/storage/vladislove2020/NP_benchmark/protenix/bench_3/workdir/3adb/input.json --out_dir results/3adb
protenix predict --input /mnt/storage/vladislove2020/NP_benchmark/protenix/bench_3/workdir/5d4s/input.json --out_dir results/5d4s
protenix predict --input /mnt/storage/vladislove2020/NP_benchmark/protenix/bench_3/workdir/6cnq/input.json --out_dir results/6cnq
protenix predict --input /mnt/storage/vladislove2020/NP_benchmark/protenix/bench_3/workdir/5wwe/input.json --out_dir results/5wwe
protenix predict --input /mnt/storage/vladislove2020/NP_benchmark/protenix/bench_3/workdir/1u1r/input.json --out_dir results/1u1r
